In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
%cd 'drive'

/content/drive


In [4]:
%cd 'MyDrive'

/content/drive/MyDrive


In [ ]:
%ls

In [6]:
%cd custom_photos/

/content/drive/My Drive/custom_photos


In [7]:
%ls

'American Crow'/                'Blue Jay'/
'American Goldfinch (Female)'/  'Common Grackle'/
'American Goldfinch (Male)'/    'House Sparrow (Female)'/
'American Robin (Adult)'/       'House Sparrow (Male)'/
'American Robin (Juvenile)'/    'House Wren'/
 BirdDetectionDeep-2.h5         'Mourning Dove'/
 BirdDetectionDeep-2.json       'Northern Cardinal (Adult Male)'/
 BirdDetectionDeep-3-best.h5    'Northern Cardinal (Female)'/
 BirdDetectionDeep-3-best.json  'Red-tailed Hawk (Dark morph)'/
 BirdDetectionDeep.h5           'Red-tailed Hawk (Light morph immature)'/
 BirdDetectionDeep.json         'Tufted Titmouse'/


In [ ]:
import os
import cv2
import glob

list_images = {}
curr_dir =  os.getcwd()
list_dir = os.listdir(curr_dir)
print(list_dir)


# for folder in list_dir:
#     print(folder)
image_array = []
for folder in list_dir:
    for img in glob.glob(str(folder)+"/*.jpg"):
      print(img)
      image = cv2.imread(img)
      image_array.append(image)
      if image is not None:
        list_images[img]=folder

In [10]:
print(image_array[0].shape)

(1024, 1024, 3)


In [11]:
image_array2 = []
for image in image_array:
  image2 = cv2.resize(image,(150,150))
  image_array2.append(image2)

In [12]:
print(image_array2[0].shape)

(150, 150, 3)


In [13]:
labels = []
for folder in list_dir:
    for img in glob.glob(str(folder)+"/*.jpg"):
      labels.append(str(folder))

In [14]:
print(labels[0])

House Wren


In [15]:
def unique_labels_words(labels):
  uniq_labels_words=[]
  for i in labels:
    if i in uniq_labels_words:
      continue
    else:
      uniq_labels_words.append(i)
  print(uniq_labels_words)
unique_labels_words(labels)

['House Wren', 'Red-tailed Hawk (Dark morph)', 'House Sparrow (Female)', 'American Robin (Adult)', 'Northern Cardinal (Adult Male)', 'American Goldfinch (Male)', 'American Goldfinch (Female)', 'American Robin (Juvenile)', 'House Sparrow (Male)', 'American Crow', 'Tufted Titmouse', 'Red-tailed Hawk (Light morph immature)', 'Blue Jay', 'Common Grackle', 'Mourning Dove', 'Northern Cardinal (Female)']


In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [17]:
def unique_labels_digits(labels):
  uniq_labels_digits=[]
  for i in labels:
    if i in uniq_labels_digits:
      continue
    else:
      uniq_labels_digits.append(i)
  print(uniq_labels_digits)
unique_labels_digits(labels)

[9, 13, 7, 3, 11, 2, 1, 4, 8, 0, 15, 14, 5, 6, 10, 12]


In [18]:
import tensorflow 
df = tensorflow.keras.utils.to_categorical(labels,num_classes=16,dtype='float32')

In [19]:
print(df)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [20]:
import numpy as np
x_train = np.array(image_array2)
y_train = df

In [21]:
print("x_dims:",x_train.shape)
print("y_dims:",y_train.shape)

x_dims: (1789, 150, 150, 3)
y_dims: (1789, 16)


In [22]:
x_train.reshape(-1,150,150,3)
print(x_train.shape)

(1789, 150, 150, 3)


In [23]:
print(x_train[0].shape)

(150, 150, 3)


In [24]:
df.shape

(1789, 16)

In [26]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2,l1_l2
from keras.utils import np_utils

In [27]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),include_top=False, weights = 'imagenet')
for layer in pre_trained_model.layers:
  layer.trainable = False

87916544/87910968 [==============================] - 1s 0us/step


In [28]:
x = Flatten()(pre_trained_model.output)
x = Dense(1024,activation = 'relu',kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4))(x)
x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
x = Dense(512,activation = 'relu')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dense(64,activation='relu')(x)
x = Dense(16 , activation = 'softmax')(x)

In [29]:
from tensorflow.keras.models import Model
model = Model(pre_trained_model.input , x)

In [30]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [31]:
model.compile(loss=categorical_crossentropy,optimizer=tensorflow.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 /10)
,metrics=['accuracy'])

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255.,rotation_range = 40,width_shift_range =0.2 ,height_shift_range = 0.2,shear_range =0.2,zoom_range = 0.2,horizontal_flip = True,validation_split=0.2)

from sklearn.model_selection import train_test_split

val_split = 0.25
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=val_split, stratify = y_train)

X_train = X_train/255.
X_val = X_val/255.


# train_generator = datagen.flow(x_train,
#                                y_train,
#                                batch_size=32,
#                                seed=0,
#                                subset='training')

# valid_generator = datagen.flow(x_train,
#                                y_train,
#                                batch_size=16,
#                                seed=0,
#                             subset='validation')

In [33]:
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

(1341, 150, 150, 3) (448, 150, 150, 3) (1341, 16) (448, 16)


In [34]:
model.fit(X_train, Y_train,
          batch_size=32,
          epochs=30,
          verbose=2,
          validation_data=(X_val, Y_val),
          shuffle=False)

Epoch 1/30
42/42 - 59s - loss: 3.9513 - accuracy: 0.3579 - val_loss: 5.3467 - val_accuracy: 0.2478
Epoch 2/30
42/42 - 53s - loss: 2.9551 - accuracy: 0.6532 - val_loss: 3.9978 - val_accuracy: 0.4464
Epoch 3/30
42/42 - 52s - loss: 2.4610 - accuracy: 0.8255 - val_loss: 3.5917 - val_accuracy: 0.5156
Epoch 4/30
42/42 - 53s - loss: 2.1847 - accuracy: 0.9120 - val_loss: 3.4760 - val_accuracy: 0.5179
Epoch 5/30
42/42 - 53s - loss: 2.0897 - accuracy: 0.9351 - val_loss: 3.4272 - val_accuracy: 0.5513
Epoch 6/30
42/42 - 53s - loss: 1.9919 - accuracy: 0.9590 - val_loss: 3.5780 - val_accuracy: 0.5379
Epoch 7/30
42/42 - 53s - loss: 1.9116 - accuracy: 0.9821 - val_loss: 3.6166 - val_accuracy: 0.5536
Epoch 8/30
42/42 - 53s - loss: 1.8718 - accuracy: 0.9918 - val_loss: 3.6488 - val_accuracy: 0.5536
Epoch 9/30
42/42 - 53s - loss: 1.8580 - accuracy: 0.9911 - val_loss: 3.8167 - val_accuracy: 0.5402
Epoch 10/30
42/42 - 53s - loss: 1.8523 - accuracy: 0.9866 - val_loss: 3.7741 - val_accuracy: 0.5379
Epoch 11/

In [35]:
'''Save the model weights into a h5 file'''
save_json = model.to_json()
with open('BirdDetectionDeep-4.json','w') as json_file:
    json_file.write(save_json )
model.save('BirdDetectionDeep-4.h5')